In [ ]:
# Use the predictions from model 1 to predict wind extremes from model 2

In [1]:
# Python ≥3.5 is required
#%reset out
import sys

assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
#from sklearn.pipeline import Pipelines
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from keras.models import Sequential
#from keras.layers.wrappers import TimeDistributed
#from keras.layers import Dense,LSTM,Conv2D, BatchNormalization,Flatten, MaxPooling2D
#from keras.layers import Conv2DTranspose,Concatenate,UpSampling2D,Cropping2D
#from keras.layers import Input, Lambda, Reshape, Dropout, Activation

from tensorflow.keras.layers import Dropout, BatchNormalization, Reshape
from tensorflow.keras.layers import Dense, Conv1D, Conv2D, Input, MaxPooling2D, Flatten, MaxPool2D, MaxPool3D, UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose, Flatten, Reshape, Cropping2D, Embedding, BatchNormalization,ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, Activation, Input, add, multiply
from tensorflow.keras.layers import ConvLSTM1D
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
from numpy import ones
import pandas as pd
#import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pathlib
import hashlib
import yaml
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# plot with cartopy
import cartopy.feature as cf
import cartopy.crs as ccrs
import matplotlib.pyplot as plt


mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# Dotenv
#from dotenv import dotenv_values
# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_plot import *
from utils.utils_unet import *
from utils.utils_resnet import *
from utils.datagenerators import *
from utils.networks import *
from utils.utils_predictions import *

2022-11-15 19:59:01.549761: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
ERROR 1: PROJ: proj_create_from_database: Open of /storage/homefs/no21h426/.conda/envs/cartopy/share/proj failed


In [2]:
#open data
conf = yaml.safe_load(open("config.yaml"))
dstot_train = xr.open_dataset('data/model1/dstot_train_day.nc')
dstot_test = xr.open_dataset('data/model1/dstot_test_day.nc')

In [3]:
# prepare into train and test
BATCH_SIZE=32
lt=1 #days in advance
dic_all = {
     'ws': None,
     'z': [1000, 850, 500, 300],              #let's use more geopotential levels
     'msl': None,
     't2m': None,
     'u': [300,925],
     'v': [300,925],
    
    # 'RMM1': None, 
    # 'RMM2': None,
    # 'amplitude': None,
    # 'phase': None,
    # 'AAO': None,
    # 'AO':None,
    # 'NAO': None,
    # 'PNA': None
    }

dg_train = DataGenerator_WeatherBench(dstot_train.sel(time=slice(f"{conf['YY_TRAIN'][0]}", f"{conf['YY_VALID']}")), 
                                     dic_all, lead_time=lt, batch_size=BATCH_SIZE, load=True)

dg_valid = DataGenerator_WeatherBench(dstot_train.sel(time=slice(f"{conf['YY_VALID']+1}", f"{conf['YY_TRAIN'][1]}")), 
                                     dic_all, lead_time=lt,  batch_size=BATCH_SIZE, mean=dg_train.mean, std=dg_train.std, load=True)
#test
dg_test = DataGenerator_WeatherBench(dstot_test, dic_all, lead_time=lt,
                                    batch_size=BATCH_SIZE, mean=dg_train.mean, std=dg_train.std, shuffle=False, load=True)

Loading data into RAM
Loading data into RAM
Loading data into RAM


In [4]:
i_shape = dg_train.data.shape[1:]
o_shape = dg_train.data.shape[1:]
output_channels = o_shape[2]

### Predict all fields with  model 1

In [5]:
output_scaling = 1
output_crop = None
models = {
          'Unet': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 4, 'dropout_rate': 0.2, 'unet_use_upsample': False, 'bottleneck': False,  'for_extremes' : False},
                    'opt_optimizer': {'lr_method': 'Constant','lr':.0001}},
          'UnetConv': {'model': 'UnetConv', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 4, 'unet_use_upsample': False, 'bottleneck': False,  'for_extremes' : False},
                   'opt_optimizer': {'lr_method': 'Constant', 'lr':.0001}},
          'resnet': {'model': 'resnet', 'run': True,
                    'opt_model': {'for_extremes' : False, 'out_channels': output_channels},
                    'opt_optimizer': {'lr_method': 'Constant','lr':.00001}}}

In [6]:
def plot_img(ax, data, time, title='', **kwargs):
    if not 'vmin' in kwargs.keys():
        mx = np.abs(data.max().values)
        kwargs['vmin'] = -mx; kwargs['vmax'] = mx
#     I = ax.imshow(data, origin='lower',  **kwargs)
    I = data.sel(time=time).plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, add_labels=False, 
                  rasterized=True, **kwargs)
    cb = fig.colorbar(I, ax=ax, orientation='horizontal', pad=0.01, shrink=0.90)
    ax.set_title(title)
    ax.coastlines(alpha=0.5)

In [7]:
var_plot = ['ws','z','msl','u','v']
level = [None, '500', None, '300', '300']

### Open model 2

In [8]:
# load data
dstot_train_xtrm = xr.open_dataset('data/model2_xtrm/dstot_train.nc')
dyex_train_xtrm = xr.open_dataset('data/model2_xtrm/dyex_train.nc')
dstot_test_xtrm = xr.open_dataset('data/model2_xtrm/dstot_test.nc')
dyex_test_xtrm = xr.open_dataset('data/model2_xtrm/dyex_test.nc')

In [9]:
dyex_train_xtrm = dyex_train_xtrm.to_array()
dyex_train_xtrm = dyex_train_xtrm.squeeze()
dyex_test_xtrm = dyex_test_xtrm.to_array()
dyex_test_xtrm = dyex_test_xtrm.squeeze()

In [10]:
# prepare datagenetor for the predictions
lead_time = 1 # I won't use lead time
dic = {
     'z': [1000, 850, 500, 300],              #let's use more geopotential levels
     'msl': None,
     't2m': None,
     'u': [300,925],
     'v': [300,925]
    # 'RMM1': None, 
    # 'RMM2': None,
    # 'amplitude': None,
    # 'phase': None,
    # 'AAO': None,
    # 'AO':None,
    # 'NAO': None,
    # 'PNA': None
    }


lt_train = DataGenerator(dstot_train_xtrm.sel(time=slice(f"{conf['YY_TRAIN'][0]}", f"{conf['YY_VALID']}")), 
                                             dyex_train_xtrm.sel(time=slice(f"{conf['YY_TRAIN'][0]}", f"{conf['YY_VALID']}")),
                                             dic, lead_time=None, batch_size=BATCH_SIZE, load=True)

lt_valid = DataGenerator(dstot_train_xtrm.sel(time=slice(f"{conf['YY_VALID']+1}", f"{conf['YY_TRAIN'][1]}")), 
                                             dyex_train_xtrm.sel(time=slice(f"{conf['YY_VALID']+1}", f"{conf['YY_TRAIN'][1]}")),
                                             dic, lead_time=None,  batch_size=BATCH_SIZE, mean=lt_train.mean, std=lt_train.std, load=True)
        #test
lt_test = DataGenerator(dstot_test_xtrm, dyex_test_xtrm, dic, lead_time=None,
                                            batch_size=BATCH_SIZE, mean=lt_train.mean, std=lt_train.std, shuffle=False, load=True)



Loading data into RAM
Loading data into RAM
Loading data into RAM


In [11]:
i2_shape = lt_train.data.shape[1:]
o2_shape = lt_train.dy.shape[1:]
mod='Unet'
print(i2_shape)
model = models['Unet']['model']
opt_model_i = models['Unet']['opt_model']
opt_optimizer_i = models['Unet']['opt_optimizer']
opt_model_new = opt_model_i.copy()
opt_model_new.update(opt_model_i)
opt_optimizer_new = opt_optimizer_i.copy()
opt_optimizer_new.update(opt_optimizer_i)
optimizer =  initiate_optimizer(lt_train, BATCH_SIZE, **opt_optimizer_new)

(34, 41, 10)


In [12]:
nam_mod_xtrm = f'ws_xtrm_{mod}.h5'
tmp_xtrm_file = pathlib.Path (f'tmp/{nam_mod_xtrm}')
print(tmp_xtrm_file)

tmp/ws_xtrm_Unet.h5


In [13]:
model_xtrm = Unet_Inputs('Unet', i2_shape, o2_shape, input_index=None, **opt_model_new)

2022-11-15 19:59:51.793506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 19:59:52.344914: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9650 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5


1
original design


In [14]:
# Something is wrong... !!!!!!!!!!!!!!!!

In [15]:
model_xtrm.model.load_weights(tmp_xtrm_file)

### Create predictions: load iterative predictions from the full model

In [16]:
# load iterative
unet_pred_iter = xr.open_dataset('predictions/daily/preds_unet_ite.nc')
unet_conv_pred_iter = xr.open_dataset('predictions/daily/preds_unetconv_int.nc')
resnet_pred_iter = xr.open_dataset('predictions/daily/preds_resnet_int.nc')

In [17]:
unet_W_pred_iter = xr.open_dataset('predictions/weekly/preds_unet_ite.nc')

In [18]:
unet_W_pred_iter

<xarray.Dataset>
Dimensions:    (lead_time: 16, time: 260, lat: 34, lon: 41, level: 5)
Coordinates:
  * lead_time  (lead_time) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
  * time       (time) datetime64[ns] 2017-01-01 2017-01-08 ... 2021-12-19
  * lat        (lat) float64 79.5 78.0 76.5 75.0 73.5 ... 34.5 33.0 31.5 30.0
  * lon        (lon) float64 -30.0 -28.5 -27.0 -25.5 ... 25.5 27.0 28.5 30.0
  * level      (level) int64 300 500 850 925 1000
Data variables:
    ws         (lead_time, time, lat, lon) float32 ...
    z          (lead_time, time, lat, lon, level) float32 ...
    msl        (lead_time, time, lat, lon) float32 ...
    t2m        (lead_time, time, lat, lon) float32 ...
    u          (lead_time, time, lat, lon, level) float32 ...
    v          (lead_time, time, lat, lon, level) float32 ...

In [19]:
#create predictions using model 1
preds_int_n = unet_pred_iter.drop('ws')

In [20]:
preds_int_n

<xarray.Dataset>
Dimensions:    (lead_time: 56, time: 1825, lat: 34, lon: 41, level: 5)
Coordinates:
  * lead_time  (lead_time) int64 1 2 3 4 5 6 7 8 9 ... 49 50 51 52 53 54 55 56
  * time       (time) datetime64[ns] 2017-01-01 2017-01-02 ... 2021-12-30
  * lat        (lat) float64 79.5 78.0 76.5 75.0 73.5 ... 34.5 33.0 31.5 30.0
  * lon        (lon) float64 -30.0 -28.5 -27.0 -25.5 ... 25.5 27.0 28.5 30.0
  * level      (level) int64 300 500 850 925 1000
Data variables:
    z          (lead_time, time, lat, lon, level) float32 ...
    msl        (lead_time, time, lat, lon) float32 ...
    t2m        (lead_time, time, lat, lon) float32 ...
    u          (lead_time, time, lat, lon, level) float32 ...
    v          (lead_time, time, lat, lon, level) float32 ...

In [22]:
# average weekly
preds_int_week = preds_int_n.resample(time='1W').mean(dim='lead_time')

ValueError: cannot rename '__resample_dim__' because it is not a variable or dimension in this dataset

In [23]:
preds_int_week

<xarray.Dataset>
Dimensions:    (lead_time: 56, lat: 34, lon: 41, level: 5, time: 262)
Coordinates:
  * lead_time  (lead_time) int64 1 2 3 4 5 6 7 8 9 ... 49 50 51 52 53 54 55 56
  * lat        (lat) float64 79.5 78.0 76.5 75.0 73.5 ... 34.5 33.0 31.5 30.0
  * lon        (lon) float64 -30.0 -28.5 -27.0 -25.5 ... 25.5 27.0 28.5 30.0
  * level      (level) int64 300 500 850 925 1000
  * time       (time) datetime64[ns] 2017-01-01 2017-01-08 ... 2022-01-02
Data variables:
    z          (time, lead_time, lat, lon, level) float32 9.09e+03 ... 148.7
    msl        (time, lead_time, lat, lon) float32 1.013e+05 ... 1.016e+05
    t2m        (time, lead_time, lat, lon) float32 281.0 281.0 ... 287.9 287.5
    u          (time, lead_time, lat, lon, level) float32 12.82 nan ... nan
    v          (time, lead_time, lat, lon, level) float32 5.456 nan ... nan

In [36]:
# prepare the data
def prepare_datainput(x,dic):
    data = []
    generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
    for var, levels in dic.items():
        if levels is None:
            data.append(x[var].expand_dims({'level': generic_level}, 1)) 
        else:
            data.append(x[var].sel(level=levels))
    data = xr.concat(data, 'level').transpose('time', 'lat', 'lon', 'level')     
    return(data)

In [37]:
lead_times = preds_int_week.lead_time

In [38]:
preds = []
for lead_time in preds_int_week.lead_time.values:
    print(lead_time)
    x = preds_int_week.sel(lead_time = lead_time)
    state = prepare_datainput(x,dic)
    p = model_xtrm.model.predict(state)
    preds.append(p)
preds = np.array(preds)

1


2022-11-15 14:08:12.900699: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201


9/9 [==============================] - 4s 52ms/step
2
9/9 [==============================] - 0s 14ms/step
3
9/9 [==============================] - 0s 14ms/step
4
9/9 [==============================] - 0s 14ms/step
5
9/9 [==============================] - 0s 14ms/step
6
9/9 [==============================] - 0s 14ms/step
7
9/9 [==============================] - 0s 16ms/step
8
9/9 [==============================] - 0s 14ms/step
9
9/9 [==============================] - 0s 14ms/step
10
9/9 [==============================] - 0s 15ms/step
11
9/9 [==============================] - 0s 14ms/step
12
9/9 [==============================] - 0s 14ms/step
13
9/9 [==============================] - 0s 14ms/step
14
9/9 [==============================] - 0s 14ms/step
15
9/9 [==============================] - 0s 14ms/step
16
9/9 [==============================] - 0s 14ms/step
17
9/9 [==============================] - 0s 14ms/step
18
9/9 [==============================] - 0s 14ms/step
19
9/9 [=============

In [41]:
lt_test.data.time

<xarray.DataArray 'time' (time: 261)>
array(['2017-01-01T00:00:00.000000000', '2017-01-08T00:00:00.000000000',
       '2017-01-15T00:00:00.000000000', ..., '2021-12-12T00:00:00.000000000',
       '2021-12-19T00:00:00.000000000', '2021-12-26T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2017-01-01 2017-01-08 ... 2021-12-26

In [43]:
preds

AttributeError: 'numpy.ndarray' object has no attribute 'time'

In [39]:
preds = preds.squeeze()
predite = (xr.DataArray(
                preds[:, :, :, :],
                dims=['lead_time', 'time', 'lat', 'lon'],
                coords={'lead_time': lead_times, 'time': lt_test.init_time, 'lat': lt_test.ds.lat, 'lon': lt_test.ds.lon},
                name='ws'
            ))

AttributeError: 'DataGenerator' object has no attribute 'init_time'

In [22]:
predite[0,:,20,10]

<xarray.DataArray 'ws' (time: 260)>
array([0.71929777, 0.99295247, 1.0943729 , 0.60210913, 0.8307347 ,
       0.8367926 , 1.5883775 , 0.89587015, 0.5037326 , 1.3937416 ,
       1.6001631 , 0.91385704, 0.8221358 , 0.788472  , 1.3461092 ,
       1.250337  , 1.2610258 , 1.5919722 , 1.3665512 , 1.0802912 ,
       1.7610247 , 1.5027195 , 1.54652   , 1.749589  , 2.0689092 ,
       1.8688372 , 1.7527364 , 1.5017422 , 1.3095716 , 1.589556  ,
       1.511946  , 1.7726723 , 1.8592132 , 1.6445208 , 1.3087138 ,
       1.2673414 , 1.4324123 , 1.6131071 , 2.0483136 , 1.6665255 ,
       1.2858539 , 1.2561951 , 1.0294578 , 1.7266532 , 0.4982098 ,
       1.1335799 , 1.3025513 , 1.5098141 , 1.0611621 , 1.0120113 ,
       0.36822847, 0.5602647 , 0.6245322 , 0.79670346, 0.85834813,
       0.76248014, 1.8706554 , 1.4884441 , 1.2519653 , 0.578849  ,
       0.6520582 , 0.8989076 , 1.855111  , 2.1822603 , 0.6026283 ,
       1.8909892 , 1.3912277 , 1.5259694 , 1.1556859 , 0.96049047,
       0.55888283, 1.3419263 , 2.1368008 , 1.680238  , 2.134949  ,
       1.1250352 , 1.7837023 , 1.694164  , 1.8917228 , 1.9701704 ,
       1.5404437 , 1.7372838 , 2.02389   , 1.3921266 , 1.6454113 ,
       1.8130031 , 1.3124423 , 1.6130234 , 1.5043372 , 1.4563109 ,
       0.7817615 , 0.82010835, 1.148043  , 1.4225286 , 0.7728847 ,
       1.2274435 , 0.5191095 , 0.58778876, 1.614323  , 1.2847993 ,
...
       0.19667807, 1.0127606 , 0.5666765 , 0.8695589 , 1.3705745 ,
       1.9204681 , 1.7467253 , 1.3992717 , 1.0081843 , 1.014684  ,
       1.2928724 , 1.817979  , 1.9691975 , 1.8365605 , 1.8839581 ,
       1.7859863 , 2.0316246 , 1.683185  , 1.9380704 , 1.7252735 ,
       1.7198362 , 1.7227634 , 1.5320046 , 1.4241462 , 1.6737987 ,
       1.5198572 , 1.5622675 , 0.9677749 , 1.054264  , 1.2703403 ,
       0.997364  , 1.3407834 , 0.8886325 , 1.8608761 , 1.2587507 ,
       0.5089939 , 0.7842756 , 0.22447385, 0.58432084, 0.99423176,
       0.96505225, 0.8837844 , 0.79220927, 1.0989516 , 1.3925432 ,
       1.3464183 , 0.78268427, 0.7926073 , 1.2222826 , 1.2498403 ,
       0.9349435 , 0.90663975, 0.53277797, 1.1257582 , 1.6809644 ,
       0.95455104, 0.33001313, 1.3731297 , 1.8861929 , 1.3071791 ,
       1.458018  , 0.76963174, 1.4142449 , 0.6943909 , 1.587566  ,
       2.0800848 , 1.5420699 , 1.8088411 , 1.871941  , 1.6371285 ,
       1.840592  , 1.666275  , 1.7396166 , 1.703854  , 1.5791306 ,
       1.4275914 , 1.6152841 , 1.7264531 , 2.0243266 , 1.8385568 ,
       0.8637106 , 1.3514278 , 0.96957135, 1.6618676 , 1.1732906 ,
       0.96141726, 0.83312863, 0.7439373 , 1.2491336 , 0.8641479 ,
       0.8931465 , 1.099752  , 1.6149141 , 1.1328142 , 1.4857836 ],
      dtype=float32)
Coordinates:
    lead_time  int64 1
  * time       (time) datetime64[ns] 2017-01-01 2017-01-08 ... 2021-12-19
    lat        float64 49.5
    lon        float64 -15.0

In [35]:
# checks
x = preds_int_n.isel(lead_time = 1)
state = prepare_datainput(x,dic)
p = model_xtrm.model.predict(state)

9/9 [==============================] - 0s 21ms/step


In [41]:
# comparison
preds_int_n.z.level

<xarray.DataArray 'level' (level: 5)>
array([ 300,  500,  850,  925, 1000])
Coordinates:
  * level    (level) int64 300 500 850 925 1000

In [42]:
# predict for 1day 
lt_test.data[:,10,10,2]


<xarray.DataArray 'z' (time: 261)>
array([-1.68448922, -0.63712742, -2.2341147 , -1.03109946, -1.77690163,
       -2.20817011, -0.35988315, -0.42472878, -2.92154946, -1.95849262,
       -1.70718941, -2.65241265, -1.2402502 , -0.84302673, -1.20782035,
       -1.58721021, -0.83817069, -0.61118636,  1.46571419, -0.47175488,
       -0.36799061, -0.03886068, -0.49120716, -0.72954335, -0.54957456,
       -0.55768202, -0.29502696, -0.41014308, -0.41987274,  0.72801596,
        0.43456028, -0.14424714, -0.27557116, -0.30638233,  0.65830023,
       -0.25125229, -1.05380318,  0.1216488 , -0.39879123, -0.43284328,
       -0.54957104, -2.16439896, -0.57551915, -0.21882595, -1.14783427,
       -2.07198303, -1.88877688, -1.89363995, -0.35177568, -1.57261748,
       -1.90661049, -1.54667642, -2.62646807, -1.94552209, -2.01524134,
       -3.54250233, -2.22438504, -1.59531416, -3.24418357, -3.09827387,
       -1.02137684, -0.10047599, -2.30058957, -1.14783779, -1.47858288,
       -1.41373021, -1.83040948, -0.22044111, -1.08947038, -1.54505422,
       -2.2924821 , -1.46074575, -1.02624343,  0.07138534,  1.25818213,
        1.06363116, -0.82357093, -0.09723864,  0.66802989,  0.32917381,
        0.63722927, -0.05183825, -0.15559548, -0.21071849, -0.26422634,
       -0.6371239 , -0.72304753, -0.73440291,  0.14434899, -1.18187928,
       -1.48345298, -1.06515151, -1.52722414, -1.93741462, -1.6536886 ,
       -0.58362661, -1.96335217, -1.66503342, -0.8333041 ,  0.74908692,
...
       -3.0447625 , -2.91181981, -2.68159459, -1.93417375, -1.36671114,
       -1.46560882, -1.065148  , -0.08750194,  0.53345796, -0.63712038,
       -0.24801141, -0.35987611, -0.54957456,  0.03571461,  0.11191562,
        0.69234876,  0.57561044, -0.17829567, -0.15721416, -0.02264927,
       -0.48958848, -0.14748802, -0.01292313, -0.31124189,  1.07173159,
        0.21081682,  0.14271975, -0.63226083, -1.30672858,  0.02436979,
       -1.31159165, -1.02948079, -0.41338396,  0.94527416, -1.57099881,
       -2.14331393, -0.92733871, -1.80608709, -1.79473523, -1.89201424,
       -1.78339393, -1.56289134, -1.9471478 , -2.07846126, -0.98084656,
       -1.26619127, -1.09271126, -2.40111296, -1.7412309 , -1.16891578,
       -1.07163326, -2.61998632, -1.58559154,  0.02760715, -2.61187885,
        0.24324315, -3.06908841, -0.49931462, -1.46074575, -0.42635449,
       -0.33394208, -0.16208427, -1.34239579, -0.73602158, -0.90463852,
        0.44915301, -0.16856953, -0.87707878, -1.00840279,  0.50427954,
        1.21117362,  0.88366237,  0.38267463,  1.10740583,  0.6485776 ,
        0.03409945,  0.6631633 ,  0.94364493,  1.76726678,  1.34087544,
        0.06490711, -0.48634409, -1.51749448, -1.57423968, -0.75223651,
       -0.3647427 , -1.07649985, -2.14655832, -1.09595565, -1.48831605,
       -1.60342514, -1.07001107, -1.62774401, -2.51784776, -0.62090898,
       -0.53173392])
Coordinates:
    level    float64 500.0
    lon      float64 -15.0
    lat      float64 64.5
  * time     (time) datetime64[ns] 2017-01-01 2017-01-08 ... 2021-12-26